In [2]:
from ultralytics import YOLO
import os
import shutil
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Load a model
model = YOLO('yolov8x-pose.pt')  # load a pretrained model

# Prompt user for the video name
video_name = input("Введите название видео (без расширения .mp4): ")

# Construct input and output file paths
input_path = f'./data/videos/input/{video_name}.mp4'
output_path = f'./data/videos/output/{video_name}_output.mp4'

# Ensure the output directory exists
os.makedirs('./data/videos/output/', exist_ok=True)

# Predict with model on the video, specifying the output directory
output_dir = './data/videos/output/'
# model.predict(source=input_path, save=True, save_txt=True, save_conf=True, project=output_dir, name=video_name+'_output', show=True, conf=0.05)
model.predict(source=input_path, save=True, save_txt=True, save_conf=True, project=output_dir, name=video_name+'_output', show=True, conf=0.05)

# Find the most recent output directory
output_dirs = sorted([d for d in os.listdir(output_dir) if os.path.isdir(os.path.join(output_dir, d))], key=lambda x: os.path.getctime(os.path.join(output_dir, x)), reverse=True)

if not output_dirs:
    print("Не найдена директория с результатами. Проверьте вывод YOLO.")
else:
    latest_output_dir = os.path.join(output_dir, output_dirs[0])

    # Find the output video file
    output_files = [f for f in os.listdir(latest_output_dir) if f.endswith('.mp4')]
    if not output_files:
        print("Не найдено видео в результатах. Проверьте вывод YOLO.")
    else:
        latest_output_file = os.path.join(latest_output_dir, output_files[0])

        # Move the output file to the desired location
        final_output_path = os.path.join(output_dir, video_name + '_output.mp4')
        shutil.move(latest_output_file, final_output_path)
        print(f"Обработанное видео сохранено по пути {final_output_path}")





WARNING  inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs


WARNING  inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks

In [3]:
import cv2

#get the dimension of the video
cap = cv2.VideoCapture('video_path')
if cap.isOpened():
    w = cap.get(cv2.CAP_PROP_FRAME_WIDTH)
    h = cap.get(cv2.CAP_PROP_FRAME_HEIGHT)
    size = (w, h)
    print(size)

In [4]:
out = cv2.VideoWriter("output.avi", cv2.VideoWriter_fourcc(*"MJPG"), cap.get(cv2.CAP_PROP_FPS), size)

NameError: name 'size' is not defined

In [5]:
import os


cap = cv2.VideoCapture(input_path)

#for webacam cv2.VideoCapture(NUM) NUM -> 0,1,2 for primary and secondary webcams..

#For saving the video file as output.avi
out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

max_file_size_mb = 20  # Максимальный размер файла в мегабайтах
max_file_size_bytes = max_file_size_mb * 1024 * 1024  # Перевод в байты

current_file_size = os.path.getsize(output_path)
while cap.isOpened() and current_file_size < max_file_size_bytes:
    ret, frame = cap.read()
    if not ret:
      break
    # Recolor Feed from RGB to BGR
    image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    #making image writeable to false improves prediction
    image.flags.writeable = False    
    
    result = model.predict(frame)    
    
    # Recolor image back to BGR for rendering
    image.flags.writeable = True   
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    # print(result.xyxy)  # img1 predictions (tensor)

    #This array will contain crops of images incase we need it 
    img_list =[]
    
    #we need some extra margin bounding box for human crops to be properly detected
    MARGIN=10

    # Print the result object to inspect its structure
    print(result)
    
    # Access the bounding boxes based on the actual structure of the result object
    for res in result:
        for bbox in res.boxes:
            print(bbox.xyxy, bbox.cls, bbox.conf)  # Adjust this line based on the actual keys in the result bject
            xmin, ymin, xmax, ymax = bbox.xyxy.cpu().numpy()[0].astype(int)
            # xmin, ymin, xmax, ymax = detection['bbox']  # Adjust this line based on the actual keys in the result object
            confidence = bbox.conf
            clas = bbox.cls

            with mp_pose.Pose(min_detection_confidence=0.3, min_tracking_confidence=0.3) as pose:
                # Media pose prediction
                results = pose.process(image[int(ymin)+MARGIN:int(ymax)+MARGIN, int(xmin)+MARGIN:int(xmax)+MARGIN])


            #Draw landmarks on image, if this thing is confusing please consider going through numpy array slicing 
            mp_drawing.draw_landmarks(image[int(ymin)+MARGIN:int(ymax)+MARGIN,int(xmin)+MARGIN:int(xmax)+MARGIN:], results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2), 
                                mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2) 
                                  ) 
            img_list.append(image[int(ymin):int(ymax),int(xmin):int(xmax):])
        # cv2_imshow(image)

        # writing in the video file 
        out.write(image)

        ## Code to quit the video incase you are using the webcam             
        # cv2.imshow('Activity recognition', image)
        # if cv2.waitKey(10) & 0xFF == ord('q'):
        #     break

cap.release()
out.release()
cv2.destroyAllWindows()

# После записи кадра обновляем текущий размер файла
current_file_size = os.path.getsize(output_path)


NameError: name 'video_path' is not defined

In [1]:
!pip freeze


absl-py==2.1.0
anyio==4.4.0
argon2-cffi==23.1.0
argon2-cffi-bindings==21.2.0
arrow==1.3.0
asttokens==2.4.1
async-lru==2.0.4
attrs==23.2.0
Babel==2.15.0
beautifulsoup4==4.12.3
bleach==6.1.0
certifi==2024.6.2
cffi==1.16.0
charset-normalizer==3.3.2
colorama==0.4.6
comm==0.2.2
contourpy==1.2.1
cvlib==0.2.7
cycler==0.12.1
debugpy==1.8.1
decorator==5.1.1
defusedxml==0.7.1
executing==2.0.1
fastjsonschema==2.20.0
filelock==3.15.3
flatbuffers==24.3.25
fonttools==4.53.0
fqdn==1.5.1
fsspec==2024.6.0
gitdb==4.0.11
GitPython==3.1.43
h11==0.14.0
httpcore==1.0.5
httpx==0.27.0
idna==3.7
imageio==2.34.2
imutils==0.5.4
intel-openmp==2021.4.0
ipykernel==6.29.4
ipython==8.25.0
ipywidgets==8.1.3
isoduration==20.11.0
jax==0.4.30
jaxlib==0.4.30
jedi==0.19.1
Jinja2==3.1.4
json5==0.9.25
jsonpointer==3.0.0
jsonschema==4.22.0
jsonschema-specifications==2023.12.1
jupyter==1.0.0
jupyter-console==6.6.3
jupyter-events==0.10.0
jupyter-lsp==2.2.5
jupyter_client==8.6.2
jupyter_core==5.7.2
jupyter_server==2.14.1
jupyter